## Part 0: Data Preprocessing

This section of the code fetches the CIFAR-10 dataset using the torchvision library and applies a series of transformations to preprocess the data. Here's a breakdown of the preprocessing steps:

1. Data Fetching:

- The CIFAR-10 dataset is downloaded and loaded using `torchvision.datasets.CIFAR10`.

2. Data Transformations:

- `ToTensor`: Converts images to PyTorch tensors
- `Normalize`: Normalizes the pixel values of each image. The pixel values are scaled to a range of [-1, 1] by subtracting the mean (0.5, 0.5, 0.5) and dividing by the standard deviation (0.5, 0.5, 0.5) for each color channel (RGB).

3. Dataset Splitting:

- The dataset is split into three subsets: training, validation, and testing.
- The proportions are set as 80% for training, 10% for validation, and 10% for testing.
- The `torch.utils.data.random_split` function is used for splitting.

4. Data Loaders:

- `DataLoader`: Wraps the datasets into iterable loaders, allowing batches of data to be efficiently fetched during training and evaluation.
- Parameters:
  - `batch_size=64`: The data is processed in mini-batches of 64 images to optimize memory usage and computational efficiency.
  - `shuffle=True` (for training): Randomly shuffles the training data to reduce model bias.
  - `shuffle=False` (for validation and testing): Maintains the order for consistent evaluation.
  

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

data_iter = iter(train_loader)
images, labels = next(data_iter)

print({'images': images.shape, 'labels': labels.shape})

Files already downloaded and verified
{'images': torch.Size([64, 3, 32, 32]), 'labels': torch.Size([64])}


## Part 1: MLP Model Architecture

This section defines, trains, and evaluates a simple Multi-Layer Perceptron (MLP) model for classifying CIFAR-10 images.

1. **Model Definition**:
   - The `MLPClassifier` class defines the architecture:
     - Three fully connected layers: 
       - `fc1`: Input layer maps 3 * 32 * 32 (32 x 32 images with 3 color channels) = 3072 pixels to 512 neurons.
       - `fc2`: Hidden layer with 128 neurons.
       - `fc3`: Output layer with 10 neurons (for the 10 classes).
     - ReLU activation is used after each layer, except the last one.

2. **Loss and Optimizer**:
   - Loss: `CrossEntropyLoss`, suitable for multi-class classification.
   - Optimizer: `Adam` optimizer with a learning rate of 0.001.

3. **Training Loop**:
   - The model is trained for 25 epochs.
   - Steps:
     - Set the model to training mode (`model.train()`).
     - Loop through batches of images and labels from `train_loader`.
     - Compute predictions, loss, and gradients, and update weights.
     - Track the running loss for monitoring.

4. **Validation**:
   - After training, the model is evaluated on the validation set (`model.eval()`).
   - Steps:
     - Loop through batches from `val_loader` without gradient computation (`torch.no_grad()`).
     - Compute predictions and compare them with ground-truth labels.
     - Calculate and print the accuracy.


In [8]:
import torch.nn as nn
import torch.optim as optim
from torch import Tensor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLPClassifier(nn.Module):
    def __init__(self):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x: Tensor) -> Tensor:
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = MLPClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, loss: {running_loss / len(train_loader)}")

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation accuracy: {100 * correct / total}")

Epoch 1, loss: 1.6668522901535034
Epoch 2, loss: 1.4576457426071168
Epoch 3, loss: 1.347080595779419
Epoch 4, loss: 1.2577714442253112
Epoch 5, loss: 1.1718513425827026
Epoch 6, loss: 1.0966192721366883
Epoch 7, loss: 1.0276023571014403
Epoch 8, loss: 0.9522481573104858
Epoch 9, loss: 0.8845983141899109
Epoch 10, loss: 0.8137466468334198
Epoch 11, loss: 0.7572565513610839
Epoch 12, loss: 0.6979521831512451
Epoch 13, loss: 0.6411679132461547
Epoch 14, loss: 0.5935547897815704
Epoch 15, loss: 0.5548017520189286
Epoch 16, loss: 0.5271416719198226
Epoch 17, loss: 0.48400299334526065
Epoch 18, loss: 0.441524991774559
Epoch 19, loss: 0.42579258465766906
Epoch 20, loss: 0.40573597185611726
Epoch 21, loss: 0.3638289365530014
Epoch 22, loss: 0.36291205077171323
Epoch 23, loss: 0.3441690725684166
Epoch 24, loss: 0.3174429668903351
Epoch 25, loss: 0.31183267146348953
Validation accuracy: 50.84
